# Abstract Base Classes & Operator Overloading

### Loading Libraries

In [87]:
# Math
import math
from math import hypot

# OS
import re
import abc
import time
import random
import bisect
import zipfile
import fnmatch
from pathlib import Path
from decimal import Decimal
from urllib.request import urlopen
from __future__ import annotations
from collections.abc import Container, Mapping
from typing import List, Protocol, NoReturn, Union, Set, Tuple, Optional, Iterable, Iterator, cast, TypeVar, Any, overload, Union, Sequence

Comparable = TypeVar('Comparable')

BaseMapping = Mapping[Comparable, Any]

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn
import matplotlib.pyplot as plt

### Creating an Abstract Base Class

In [42]:
class MediaLoader(abc.ABC):
    @abc.abstractmethod
    def play(self) -> None:
        ...

    @property
    @abc.abstractmethod
    def ext(self) -> str:
        ...

In [43]:
MediaLoader.__abstractmethods__

frozenset({'ext', 'play'})

In [44]:
class Wav(MediaLoader):
    pass

In [45]:
x = Wav()

TypeError: Can't instantiate abstract class Wav with abstract methods ext, play

In [46]:
class Ogg(MediaLoader):
    ext = '.ogg'
    def play(self):
        pass

In [47]:
o = Ogg()

### The ABC of Collections

In [48]:
Container.__abstractmethods__

frozenset({'__contains__'})

In [49]:
help(Container.__contains__)

Help on function __contains__ in module collections.abc:

__contains__(self, x)



In [50]:
class OddIntegers:
    def __contains__(self, x: int) -> bool:
        return x % 2 != 0

In [51]:
odd = OddIntegers()

In [52]:
isinstance(odd, Container)

True

In [53]:
issubclass(OddIntegers, Container)

True

In [54]:
odd = OddIntegers()

In [55]:
1 in odd

True

In [56]:
2 in odd

False

In [57]:
3 in odd

True

### The `collections.abc` Module

In [59]:
Comparable = TypeVar('Comparable')

BaseMapping = Mapping[Comparable, Any]

In [60]:
x = dict({"a": 42, "b": 7, "c": 6})

y = dict([("a", 42), ("b", 7), ("c", 6)])

In [61]:
x == y

True

In [66]:
BaseMapping = Mapping[Comparable, Any]
class Lookup(BaseMapping):
    @overload
    def __init__(self, source: Iterable[tuple[Comparable, Any]]) -> None:
        ...

    @overload
    def __init__(self, source: BaseMapping) -> None:
        ...

    def __init__(self, source: Union[Iterable[tuple[Comparable, Any]], BaseMapping, None] = None,) ->None:
        sorted_pairs: Sequence[tuple[Comparable, Any]]
        if isinstance(source, Sequence):
            sorted_pairs = sorted(source)
        elif isinstance(source, abc.Mapping):
            sorted_pairs = sorted(source.items())
        else:
            sorted_pairs = []
        self.key_list = [p[0] for p in sorted_pairs]
        self.value_list = [p[1] for p in sorted_pairs]

    def __len__(self) -> int:
        return len(self.key_list)

    def __iter__(self) -> Iterator[Comparable]:
        return iter(self.key_list)

    def __contains__(self, key: object) -> bool:
        index = bisect.bisect_left(self.key_list, key)
        return key == self.key_list[index]

    def __getitem__(self, key: Comparable) -> Any:
        index = bisect.bisect_left(self.key_list, key)
        if key == self.key_list[index]:
            return self.value_list[index]
        raise KeyError(key)

In [71]:
x = Lookup(
    [
        ["z", "Zillah"],
        ["a", "Amy"],
        ["c", "Clara"],
        ["b", "Basil"],
    ]
)

In [74]:
x["c"]

'Clara'

In [75]:
x["m"] = "Maud"

TypeError: 'Lookup' object does not support item assignment

In [76]:
class Comparable(Protocol):
    def __eq__(self, other:Any) -> bool: ...
    def __ne__(self, other:Any) -> bool: ...
    def __le__(self, other:Any) -> bool: ...
    def __lt__(self, other:Any) -> bool: ...
    def __ge__(self, other:Any) -> bool: ...
    def __gt__(self, other:Any) -> bool: ...

In [77]:
class Die (abc.ABC):
    def __init__(self) -> None:
        self.face: int
        self.roll()

    @abc.abstractmethod
    def roll(self) -> None:
        ...

    def __repr__(self) -> str:
        return f"{self.face}"

In [78]:
"""
Messy Code Sample as follow:
"""

# class Bad(Die):
#     def roll(self, a: int, b: int) -> float:
#         return (a+b)/2

'\nMessy Code Sample as follow:\n'

In [79]:
class D4(Die):
    def roll(self) -> None:
        self.face = random.choice((1, 2, 3, 4))

In [80]:
class D6(Die):
    def roll(self) -> None:
        self.face = random.randint(1, 6)

In [81]:
class Dice(abc.ABC):
    def __init__(self, n: int, die_class: Type[Die]) -> None:
        self.dice = [die_class() for _ in range(n)]

    @abc.abstractmethod
    def roll(self) -> None:
        ...

    @property
    def total(self) -> int:
        return sum(d.face for d in self.dice)

In [85]:
class SimpleDice(Dice):
    def roll(self) -> None:
        for d in self.dice:
            d.roll()

In [88]:
sd = SimpleDice(6, D6)

In [89]:
sd.roll()

In [90]:
sd.total

18

In [91]:
class YatchDice(Dice):
    def __init__(self) -> None:
        super().__init__(5, D6)
        self.saved: Set[int] = set()

    def saving(self, positions: Iterable[int]) -> "YatchDice":
        if not all(0 <= n < 6 for n in positions):
            raise ValueError("Invalid position")
        self.saved = set(positions)
        return self

    def roll(self) -> None:
        for n, d in enumerate(self.dice):
            if n not in self.saved:
                d.roll()
        self.saved = set()

In [92]:
sd = YatchDice()

In [93]:
sd.roll()

In [94]:
sd.dice

[6, 3, 5, 1, 6]

In [95]:
sd.saving([0, 1, 2]).roll()

In [96]:
sd.dice

[6, 3, 5, 2, 2]